<a href="https://colab.research.google.com/github/tejask666/Tejas_INFO5731_Spring2020/blob/master/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [2019 Dell labtop](https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/dp/B07N49F51N/ref=sr_1_11?crid=1IJ7UWF2F4GHH&keywords=dell%2Bxps%2B15&qid=1580173569&sprefix=dell%2Caps%2C181&sr=8-11&th=1) on amazon.

(2) Collect the top 100 User Reviews of the film [Joker](https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv) from IMDB.

(3) Collect the abstracts of the top 100 research papers by using the query [natural language processing](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(4) Collect the top 100 tweets by using hashtag ["#wuhancoronovirus"](https://twitter.com/hashtag/wuhancoronovirus) from Twitter. 


# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [0]:
    # set options to be headless, ..
    !apt-get update
    import os
    !apt install chromium-chromedriver
    !cp /usr/lib/chromium-browser/chromedriver /usr/bin
    !pip install selenium
    from selenium import webdriver
    from selenium.webdriver import ActionChains
    from selenium.webdriver.support.wait import WebDriverWait
    from selenium.webdriver.support import expected_conditions as ec
    import time
    import spacy
    from spacy import displacy
    import benepar
    from pandas import DataFrame
    import nltk
    from nltk.stem import WordNetLemmatizer
    from nltk.stem import PorterStemmer
    from nltk.tokenize import word_tokenize
    from google.colab import files
    
    class Assignment1():
        def get_data_in_csv(self):
             nltk.download('punkt')
             nltk.download('wordnet')
             options = webdriver.ChromeOptions()
             options.add_argument('--headless')
             options.add_argument('--no-sandbox')
             options.add_argument('--disable-dev-shm-usage')
             driver = webdriver.Chrome('chromedriver',options=options)
             driver.get("https://www.imdb.com/title/tt7286456/")
             driver.find_element_by_xpath("//a[text()='USER REVIEWS']").click()
             for i in range(0, 3):
                time.sleep(15)
                loadmorebutton = driver.find_element_by_xpath("//button[@id='load-more-trigger']")
                driver.execute_script("arguments[0].scrollIntoView();", loadmorebutton)
                oActions = ActionChains(driver).move_to_element(loadmorebutton)
                oActions.perform()
                loadmorebutton.click()
                time.sleep(5)
             list = driver.find_elements_by_xpath('//div[@class="review-container"]')
             for i in range(0,100):
                driver.execute_script("arguments[0].scrollIntoView();", list[i])
                wait = WebDriverWait(driver, 10)
                wait.until(ec.visibility_of(list[i]))
                content = list[i].text
                A = Assignment1()
                A.writeintocsv(content, i)
    
        def writeintocsv(self, text, i):
            A = Assignment1()
            removedpun = A.removeallpunctuation(text)
            cleaned_data = A.lemma_and_stem(removedpun)
            dataset = {'data': text, 'cleaned_data': cleaned_data}
            df = DataFrame(dataset, columns=['data', 'cleaned_data'], index=[i])
            export_csv = df.loc[[i]].to_csv('testit.csv', index=None,header=0,mode='a')  
    
        def removeallpunctuation(self, textstring):
            textstring = textstring.lower()
            temp = ''
            removetext = '''!()-[]{};:'"\,<>./?@#$%^&*_~0123456789'''
            for i in range(len(textstring)):
                if textstring[i] in removetext:
                    pass
                else:
                    temp = temp + textstring[i]
            return temp
    
        def cleansentence(self, sentence):
            temp = ''
            textlist = word_tokenize(sentence)
            stopwordlist = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself",
                            "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its",
                            "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom",
                            "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being",
                            "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but",
                            "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about",
                            "against", "between", "into", "through", "during", "before", "after", "above", "below", "to",
                            "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then",
                            "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few",
                            "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so",
                            "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
            for i in range(len(textlist)):
                if textlist[i] in stopwordlist:
                    pass
                else:
                    temp = temp + " " + textlist[i]
            return temp
    
        def lemma_and_stem(self,sentence):
            temp=''
            A=Assignment1()
            text=A.cleansentence(sentence)
            nlp = spacy.load("en_core_web_sm")
            doc = nlp(text)
            for token in doc:
                ps = PorterStemmer()
                word=ps.stem(token.lemma_)
                temp = temp + " " + word
            return temp
    
        def count_named_entity_recognition(self,text):
            person,org,location,product,date=0,0,0,0,0
            nlp = spacy.load('en_core_web_sm')
            doc = nlp(text)
            for ent in doc.ents:
                if ent.label_=='PERSON':
                    person=person+1
                elif ent.label_=='ORG':
                    org=org+1
                elif ent.label_=='LOC' or ent.label_=='GPE':
                    location=location+1
                elif ent.label_=='PRODUCT':
                    product=product+1
                elif ent.label_=='DATE':
                    date=date+1
            print("Total number of persons: "+str(person)+",organization: "+str(org)+",location: "+str(location)+",product: "+str(product)+",date:"+str(date))
    
        def tagpartofspeech(self,cleanedtext):
                nlp = spacy.load("en_core_web_sm")
                doc = nlp(cleanedtext)
                noun_count,verb_count,adverb_count,adj_count=0,0,0,0
                for token in doc:
                   if token.pos_ == 'NOUN':
                    noun_count=noun_count+1
                   elif token.pos_ == 'ADJ':
                       adj_count=adj_count+1
                   elif token.pos_ == 'VERB':
                       verb_count=verb_count+1
                   elif token.pos_ == 'ADV':
                       adverb_count=adverb_count+1
                print("Total number of nouns: "+str(noun_count)+",adjective: "+str(adj_count)+",adverb: "+str(adverb_count)+",verb: "+str(verb_count))
        def contituencyparse(self,sentence):
                #nltk.download('punkt')
                benepar.download('benepar_en2')
                parser = benepar.Parser("benepar_en2")
                tree = parser.parse(sentence)
                print("Contituency parsed representation of sentence:")
                print(tree)
        def dependencyparse(self,sentence):
                nlp = spacy.load('en')
                doc = nlp(sentence)
                print("Dependecy parsed representation of sentence:")
                for token in doc:
                  print("{0}/{1} <--{2}-- {3}/{4}".format(token.text, token.tag_, token.dep_, token.head.text, token.head.tag_))
                displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})
    
    A = Assignment1()
    A.get_data_in_csv()
    A.count_named_entity_recognition("one good comic book movie perform phoenix make speechless end")
    A.tagpartofspeech("one good comic book movie perform phoenix make speechless end")
    A.contituencyparse("one good comic book movie perform phoenix make speechless end")
    A.dependencyparse("one good comic book movie perform phoenix make speechless end")
    

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (150 kB/s)
Reading package lists... Done
Reading package lists... Done
Building d

**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

**Answer -**

**Constituency Parsing Tree:**


*  Constituency parsing aims to extract a constituency-based parse from a sentence that represents its syntactic structure according to a phrase structure grammar.
*  By using Constituency parsing i have separated out Nouns and Verbs Phrases from the sentences.


*   After separating out Nouns and Verb phrases,further separation is done i.e "Verbs and Nouns" are separated from verb phrase.
*   I have used nltk i.e. a processing class for deriving trees that represent possible structures for a sequence of tokens. These tree structures are known as “parses”. Typically, parsers are used to derive syntax trees for sentences. 

**Dependency Parsing Tree:**



*   Dependency parsing is the process of extracting the dependency parse of a sentence to represent its grammatical structure. It defines the dependency relationship between headwords and their dependents. The head of a sentence has no dependency and is called the root of the sentence
By using dependency parsing i have delineated the dependency between a word (such as a verb) and the phrases which builds upon such as the subject and object phrases of that verb.


*   I have used spacy library for dependency parsing.



For both the above parser, i have used benepar parser.
